<a href="https://colab.research.google.com/github/nitin7447/Big-Data-Practical/blob/main/NitinKumavat_25015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 13.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=fd36d33f94054146a126aed1d0345ef1864242b96d5515fc22f99400b5956994
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler,MinMaxScaler

In [3]:
spark=SparkSession.builder.master("local[4]")\
    .appName('NitinExam').getOrCreate()

In [7]:
data=spark.read.csv("wine-class.csv",header=True)

In [9]:
data.show(5)

+-----+-------+-----------+----+------------------+----------+--------------+-----------+---------------------+----------------+----------------+----+-----------------------------+--------+
|class|alcohol| malic_acid| ash| alcalinity_of_ash| magnesium| total_phenols| flavanoids| nonflavanoid_phenols| proanthocyanins| color_intensity| hue| od280/od315_of_diluted_wines| proline|
+-----+-------+-----------+----+------------------+----------+--------------+-----------+---------------------+----------------+----------------+----+-----------------------------+--------+
|    1|  14.23|       1.71|2.43|              15.6|       127|           2.8|       3.06|                 0.28|            2.29|            5.64|1.04|                         3.92|    1065|
|    1|   13.2|       1.78|2.14|              11.2|       100|          2.65|       2.76|                 0.26|            1.28|            4.38|1.05|                          3.4|    1050|
|    1|  13.16|       2.36|2.67|              18.6

In [10]:
data

DataFrame[class: string, alcohol: string,  malic_acid: string,  ash: string,  alcalinity_of_ash: string,  magnesium: string,  total_phenols: string,  flavanoids: string,  nonflavanoid_phenols: string,  proanthocyanins: string,  color_intensity: string,  hue: string,  od280/od315_of_diluted_wines: string,  proline: string]

In [45]:
data.summary().show()

+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+---------------------+------------------+-----------------+-------------------+-----------------------------+-----------------+
|summary|             class|           alcohol|        malic_acid|               ash| alcalinity_of_ash|         magnesium|     total_phenols|        flavanoids| nonflavanoid_phenols|   proanthocyanins|  color_intensity|                hue| od280/od315_of_diluted_wines|          proline|
+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+---------------------+------------------+-----------------+-------------------+-----------------------------+-----------------+
|  count|               178|               178|               178|               178|               178|               178|          

In [40]:
data.groupby('class').count().show()

+-----+-----+
|class|count|
+-----+-----+
|    3|   48|
|    1|   59|
|    2|   71|
+-----+-----+



In [47]:
data.describe()

DataFrame[summary: string, class: string, alcohol: string,  malic_acid: string,  ash: string,  alcalinity_of_ash: string,  magnesium: string,  total_phenols: string,  flavanoids: string,  nonflavanoid_phenols: string,  proanthocyanins: string,  color_intensity: string,  hue: string,  od280/od315_of_diluted_wines: string,  proline: string]

In [48]:
pdf=data.toPandas()

In [49]:
pdf.isnull().sum()

class                            0
alcohol                          0
 malic_acid                      0
 ash                             0
 alcalinity_of_ash               0
 magnesium                       0
 total_phenols                   0
 flavanoids                      0
 nonflavanoid_phenols            0
 proanthocyanins                 0
 color_intensity                 0
 hue                             0
 od280/od315_of_diluted_wines    0
 proline                         0
dtype: int64

In [50]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [58]:
input_cols = data.columns[1:]

In [59]:
data.select(input_cols).show()

+-------+-----------+----+------------------+----------+--------------+-----------+---------------------+----------------+----------------+----+-----------------------------+--------+
|alcohol| malic_acid| ash| alcalinity_of_ash| magnesium| total_phenols| flavanoids| nonflavanoid_phenols| proanthocyanins| color_intensity| hue| od280/od315_of_diluted_wines| proline|
+-------+-----------+----+------------------+----------+--------------+-----------+---------------------+----------------+----------------+----+-----------------------------+--------+
|  14.23|       1.71|2.43|              15.6|       127|           2.8|       3.06|                 0.28|            2.29|            5.64|1.04|                         3.92|    1065|
|   13.2|       1.78|2.14|              11.2|       100|          2.65|       2.76|                 0.26|            1.28|            4.38|1.05|                          3.4|    1050|
|  13.16|       2.36|2.67|              18.6|       101|           2.8|       3.

In [61]:
from pyspark.ml.feature import Imputer, MinMaxScaler

In [64]:
imputed_col=['f_{}'.format(i+1) for i in range(13)]

In [65]:
imputed_col

['f_1',
 'f_2',
 'f_3',
 'f_4',
 'f_5',
 'f_6',
 'f_7',
 'f_8',
 'f_9',
 'f_10',
 'f_11',
 'f_12',
 'f_13']

In [68]:
for col in data.columns:
  data=data.withColumn(col,data[col].cast('double'))

In [69]:
model=Imputer(strategy='mean',
              missingValue=None,
              inputCols=input_cols,
              outputCols=imputed_col).fit(data)

In [71]:
impute_data=model.transform(data)

In [72]:
impute_data.show()

+-----+-------+-----------+----+------------------+----------+--------------+-----------+---------------------+----------------+----------------+----+-----------------------------+--------+-----+----+----+----+-----+----+----+----+----+----+----+----+------+
|class|alcohol| malic_acid| ash| alcalinity_of_ash| magnesium| total_phenols| flavanoids| nonflavanoid_phenols| proanthocyanins| color_intensity| hue| od280/od315_of_diluted_wines| proline|  f_1| f_2| f_3| f_4|  f_5| f_6| f_7| f_8| f_9|f_10|f_11|f_12|  f_13|
+-----+-------+-----------+----+------------------+----------+--------------+-----------+---------------------+----------------+----------------+----+-----------------------------+--------+-----+----+----+----+-----+----+----+----+----+----+----+----+------+
|  1.0|  14.23|       1.71|2.43|              15.6|     127.0|           2.8|       3.06|                 0.28|            2.29|            5.64|1.04|                         3.92|  1065.0|14.23|1.71|2.43|15.6|127.0| 2.8|3.

In [73]:
assemble=VectorAssembler(inputCols=imputed_col,
                        outputCol='assembled_features')

In [74]:
dat=assemble.transform(impute_data)

In [75]:
dat.show(5)

+-----+-------+-----------+----+------------------+----------+--------------+-----------+---------------------+----------------+----------------+----+-----------------------------+--------+-----+----+----+----+-----+----+----+----+----+----+----+----+------+--------------------+
|class|alcohol| malic_acid| ash| alcalinity_of_ash| magnesium| total_phenols| flavanoids| nonflavanoid_phenols| proanthocyanins| color_intensity| hue| od280/od315_of_diluted_wines| proline|  f_1| f_2| f_3| f_4|  f_5| f_6| f_7| f_8| f_9|f_10|f_11|f_12|  f_13|  assembled_features|
+-----+-------+-----------+----+------------------+----------+--------------+-----------+---------------------+----------------+----------------+----+-----------------------------+--------+-----+----+----+----+-----+----+----+----+----+----+----+----+------+--------------------+
|  1.0|  14.23|       1.71|2.43|              15.6|     127.0|           2.8|       3.06|                 0.28|            2.29|            5.64|1.04|          

In [76]:
scaler=MinMaxScaler(min=0,max=1.0,
                    inputCol='assembled_features',
                    outputCol='features')

In [77]:
sc=scaler.fit(dat).transform(dat)

In [78]:
sc.select('features').show(5)

+--------------------+
|            features|
+--------------------+
|[0.84210526315789...|
|[0.57105263157894...|
|[0.56052631578947...|
|[0.87894736842105...|
|[0.58157894736842...|
+--------------------+
only showing top 5 rows

